Test

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
from sklearn.metrics import accuracy_score

directory = os.getcwd()
#set path for input DATA
data_path = os.path.join(directory, "data")
print(data_path)


artists_file = os.path.join(data_path, "artists.dat")
artists_spotify_file = os.path.join(data_path, "data_by_artist_o.csv")
tag_file = os.path.join(data_path, "tags.dat")
users_artists_file = os.path.join(data_path, "user_artists.dat")
users_tags_file = os.path.join(data_path, "user_taggedartists.dat")




/Users/luis/Documents/21_ISYE6740_CDA/Project/Code/data


In [3]:
#import data
artists_spotify_data = pd.read_csv(artists_spotify_file)
print("artists_spotify_data shape: ",artists_spotify_data.shape)

artists_data = pd.read_csv(artists_file,sep="\t")
print("artists_data shape: ",artists_data.shape)

tag_data = pd.read_csv(tag_file,sep="\t")
print("tag_data shape: ",tag_data.shape)


users_artists_data = pd.read_csv(users_artists_file,sep="\t")
print("users_artists_data shape: ",users_artists_data.shape)


users_tags_data = pd.read_csv(users_tags_file,sep="\t")
print("users_tags_data shape: ",users_tags_data.shape)




artists_spotify_data shape:  (28680, 16)
artists_data shape:  (17632, 4)
tag_data shape:  (11946, 2)
users_artists_data shape:  (92834, 3)
users_tags_data shape:  (186479, 6)


In [4]:
#listening habits of artists. Weight = scrobbles = plays
users_artists_data

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983
...,...,...,...
92829,2100,18726,337
92830,2100,18727,297
92831,2100,18728,281
92832,2100,18729,280


In [5]:
#Sparse matrix, we need to pivot the data
data_pivot = users_artists_data.pivot(index='userID',
                                       columns='artistID').fillna(0).astype(int)
data_pivot.head()

weight                                                        ...  \
artistID  1     2     3     4     5     6     7     8     9     10     ...   
userID                                                                 ...   
2             0     0     0     0     0     0     0     0     0     0  ...   
3             0     0     0     0     0     0     0     0     0     0  ...   
4             0     0     0     0     0     0     0     0     0     0  ...   
5             0     0     0     0     0     0     0     0     0     0  ...   
6             0     0     0     0     0     0     0     0     0     0  ...   

                                                                      
artistID 18736 18737 18738 18739 18740 18741 18742 18743 18744 18745  
userID                                                                
2            0     0     0     0     0     0     0     0     0     0  
3            0     0     0     0     0     0     0     0     0     0  
4            0     0     0     0     0     0     0     0     0     0  
5            0     0     0     0     0     0     0     0     0     0  
6            0     0     0     0     0     0     0     0     0     0  

[5 rows x 17632 columns]

In [6]:
#Trying to make it more memory efficient
from scipy import sparse
# create scipy sparse from pivot tables
data_sparse = sparse.csr_matrix(data_pivot)

In [13]:
#Quality Check


#user 2100, 3rd artist has 408 scrobbles
print(data_pivot)
print('\n')

#fist five artist scrobbles for user 2100
print (users_artists_data[users_artists_data['userID']==2100].head(5))
print('\n')


#user ID 2100 is now in the 1892th position
print('data_sparse shape',data_sparse.shape)
print('\n')

#ok 3rd artist (index 2) has 408 plays, 6th artist (index 5) has 404 plays
print(data_sparse[1891]) #index is 1892-1


         weight                                                        ...  \
artistID  1     2     3     4     5     6     7     8     9     10     ...   
userID                                                                 ...   
2             0     0     0     0     0     0     0     0     0     0  ...   
3             0     0     0     0     0     0     0     0     0     0  ...   
4             0     0     0     0     0     0     0     0     0     0  ...   
5             0     0     0     0     0     0     0     0     0     0  ...   
6             0     0     0     0     0     0     0     0     0     0  ...   
...         ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
2095          0     0     0     0     0     0     0     0     0     0  ...   
2096          0     0     0     0     0     0     0     0     0     0  ...   
2097          0     0     0     0     0     0     0     0     0     0  ...   
2099          0     0     0     0     0     0     0     0     0 

In [14]:
#Merging some tables 
df_scrobbles = users_artists_data.merge(artists_data[['id','name']],
                               how = 'inner',
                               left_on = 'artistID',
                               right_on = 'id')


df_scrobbles = df_scrobbles.drop(columns=['id'])
df_scrobbles.columns = ['userID','artistID','scrobbles','name']

print(df_scrobbles.shape)
df_scrobbles.head()

(92834, 4)


,userID,artistID,scrobbles,name
0,2,51,13883,Duran Duran
1,4,51,228,Duran Duran
2,27,51,85,Duran Duran
3,28,51,10,Duran Duran
4,62,51,528,Duran Duran


In [16]:
#inner join with spotify data. 
#ISSUE: not all artists are found on spotify dataset
#this might be ok. because we will create synthetic artists
print(artists_data.shape)
df = artists_data.merge(artists_spotify_data,
                               how = 'inner',
                               left_on = artists_data['name'].str.lower(),
                               right_on = artists_spotify_data['artists'].str.lower())

print(df.shape)
df.head()

(17632, 4)
(4878, 21)


,key_0,id,name,url,pictureURL,genres,artists,acousticness,danceability,duration_ms,...,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,marilyn manson,7,Marilyn Manson,http://www.last.fm/music/Marilyn+Manson,http://userserve-ak.last.fm/serve/252/2558217.jpg,"['alternative metal', 'industrial', 'industria...",Marilyn Manson,0.016293,0.488770,253835.121622,...,0.127704,0.301474,-6.852716,0.086965,140.086203,0.335959,47.162162,9,1,148
1,combichrist,9,Combichrist,http://www.last.fm/music/Combichrist,http://userserve-ak.last.fm/serve/252/51273485...,"['aggrotech', 'ebm', 'electro-industrial', 'in...",Combichrist,0.000003,0.492000,285667.000000,...,0.756000,0.319000,-5.970000,0.043300,120.036000,0.212000,45.000000,1,1,2
2,dimmu borgir,15,Dimmu Borgir,http://www.last.fm/music/Dimmu+Borgir,http://userserve-ak.last.fm/serve/252/52216127...,"['black metal', 'melodic death metal', 'metal'...",Dimmu Borgir,0.000166,0.193333,272549.333333,...,0.655333,0.219400,-3.887000,0.116267,109.019000,0.160333,47.666667,6,0,6
3,cradle of filth,25,Cradle of Filth,http://www.last.fm/music/Cradle+of+Filth,http://userserve-ak.last.fm/serve/252/30799583...,"['alternative metal', 'black metal', 'british ...",Cradle Of Filth,0.001335,0.402750,378683.750000,...,0.199202,0.111675,-3.789750,0.122325,129.148250,0.153450,48.750000,2,0,8
4,kmfdm,36,KMFDM,http://www.last.fm/music/KMFDM,http://userserve-ak.last.fm/serve/252/19970449...,"['cyberpunk', 'ebm', 'electro-industrial', 'in...",KMFDM,0.000215,0.575100,305090.800000,...,0.507929,0.318270,-5.290500,0.065230,127.084400,0.431600,41.100000,6,0,20


In [18]:
#left join - looks like there might be 6 duplicates??
print(artists_data.shape)
df = artists_data.merge(artists_spotify_data,
                               how = 'left',
                               left_on = artists_data['name'].str.lower(),
                               right_on = artists_spotify_data['artists'].str.lower())

print(df.shape)
df.head()

(17632, 4)
(17638, 21)


,key_0,id,name,url,pictureURL,genres,artists,acousticness,danceability,duration_ms,...,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
0,malice mizer,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,diary of dreams,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,carpathian forest,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,moi dix mois,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bella morte,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#test_merge.to_excel('ttest.xlsx', sheet_name='data', index = False)

In [21]:
#Potential Risk
#some users play lots of music. suppose their top artist was playe 10,000 times. 
#other users might be similar in preferences, with the same top artist, but with only 100 plays
#CAN WE NORMALIZE AT USER LEVEL? 
scrobbles_sorted = df_scrobbles.groupby('userID').agg({'scrobbles': ['min', 
                                                                  'max',
                                                                  'sum',
                                                                  'count']})

scrobbles_sorted
#for example. For userID 2: 13,383 represents their top artist
#for userID 6: 1,011 represet their top artist. If both top artists are Duran Duran
#then the data points wont be as close to each other when maybe they should???
#but if we scale the whole dataset the problem will remain suppose 13,383 gets converted
#to 1.383 and 1,011 gets converted to 0.101. 






scrobbles                     
             min    max     sum count
userID                               
2           1315  13883  168737    50
3             65  13176   20501    50
4            193   4983   26930    50
5            115    884   13159    50
6             13     41    1011    50
...          ...    ...     ...   ...
2095          31    548    4885    50
2096         336  14690   62714    50
2097         107   2628   13830    50
2099          72    770   10179    50
2100         228   2026   30877    50

[1892 rows x 4 columns]

In [8]:
#new